In [196]:
import pandas as pd
import numpy as np

datapath="../data/"

submission_data = pd.read_csv(datapath + "submission.csv")
test_data = pd.read_csv(datapath + "test.csv")
train_data = pd.read_csv(datapath + "train.csv")
countrycodes=pd.read_csv(datapath+"country-codes_csv.csv")
indicator_data=pd.read_excel(datapath+"Indicator_list.xlsx").fillna("")

indicator_data["long_form"]=indicator_data["long_form"].apply(lambda x: "_".join(x.split(" ")))
train_data[["ConfirmedCases","Fatalities"]]=train_data[["ConfirmedCases","Fatalities"]].astype(int)
submission_data[["ConfirmedCases","Fatalities"]]=submission_data[["ConfirmedCases","Fatalities"]].astype(int)

In [22]:
train_data

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0,0
1,2,NaN,Afghanistan,2020-01-23,0,0
2,3,NaN,Afghanistan,2020-01-24,0,0
3,4,NaN,Afghanistan,2020-01-25,0,0
4,5,NaN,Afghanistan,2020-01-26,0,0
...,...,...,...,...,...,...
19105,29361,NaN,Zimbabwe,2020-03-22,3,0
19106,29362,NaN,Zimbabwe,2020-03-23,3,1
19107,29363,NaN,Zimbabwe,2020-03-24,3,1
19108,29364,NaN,Zimbabwe,2020-03-25,3,1


In [256]:
import requests
from tqdm import tqdm

def import_data(indicator="",indicator_type="",name=""):
    url="https://ghoapi.azureedge.net/api/{}".format(indicator)
    data = pd.DataFrame(requests.get(url).json()["value"])
    data.drop([col for col in data.columns if data[col].isna().sum() == data.shape[0]],axis=1,inplace=True)
    data=data.drop(data[data.NumericValue.isna()].index,axis=0).reset_index(drop=True)
    data=data[data.Dim1==indicator_type] if len(indicator_type)>0 else data
    data=data[["IndicatorCode","SpatialDim","TimeDim","NumericValue"]].drop_duplicates(subset=["IndicatorCode","SpatialDim"],keep="last").reset_index(drop=True)
    data=data[["SpatialDim","NumericValue"]]
    data.columns=["Shortcut",name]
    
    return data

def import_who_indicators(indicator_data):
    data=[import_data(indicator["short_form"],indicator["addition"],indicator["long_form"]) for _,indicator in tqdm(indicator_data.iterrows())]
    return data
    
who_data_list=import_who_indicators(indicator_data)


0it [00:00, ?it/s]
1it [00:01,  1.23s/it]
2it [00:01,  1.01s/it]
3it [00:02,  1.02s/it]
4it [00:03,  1.12it/s]
5it [00:03,  1.30it/s]
6it [00:04,  1.53it/s]
7it [00:04,  1.58it/s]
8it [00:07,  1.31s/it]
9it [00:10,  1.85s/it]
10it [00:13,  2.19s/it]
11it [00:14,  1.30s/it]


In [257]:
def match_country_code(countrycodes,traindata):
    countrycodes=countrycodes[["ISO3166-1-Alpha-3","official_name_en"]]
    countrycodes=countrycodes.drop(countrycodes[countrycodes["ISO3166-1-Alpha-3"].isna()].index,axis=0).reset_index(drop=True)
    countrycodes.columns = ["Shortcut", "Country_Region"]
    result = traindata.merge(countrycodes,on="Country_Region",how="left").reset_index(drop=True)
    return result

new_traindata = match_country_code(countrycodes,train_data)

def match_dataframes(new_traindata,who_data):
    result = new_traindata.merge(who_data,on="Shortcut",how="left")
    return result

test_traindata = new_traindata.copy()
for who_data in tqdm(who_data_list):
    test_traindata=match_dataframes(test_traindata,who_data)


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 56.73it/s]


In [259]:
test_traindata.columns

Index(['Id', 'Province_State', 'Country_Region', 'Date', 'ConfirmedCases',
       'Fatalities', 'Shortcut',
       'Population_using_at_least_basic_sanitation_services_(%)',
       'Hospital_beds_(per_10_000_population)',
       'Total_expenditure_on_health_as_a_percentage_of_gross_domestic_product',
       'Medical_doctors_(per_10_000_population)',
       'Nursing_and_midwifery_personnel_(per_10_000_population)',
       'Skilled_health_professionals_density_(per_10_000_population)',
       'Prevalence_of_smoking_any_tobacco_product_among_persons_aged_>=_15_years',
       'Raised_fasting_blood_glucose_(>=7.0_mmol/L_or_on_medication)(age-standardized_estimate)',
       'Raised_blood_pressure_(SBP>=140_OR_DBP>=90)_(age-standardized_estimate)',
       'Prevalence_of_obesity_among_adults,_BMI_>=_30_(age-standardized_estimate)_(%)',
       'Health_Service_Provision'],
      dtype='object')

In [254]:
new_traindata[new_traindata["Shortcut"].isna()]["Country_Region"].unique()

array(['Bolivia', 'Brunei', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       "Cote d'Ivoire", 'Diamond Princess', 'Eswatini', 'Iran',
       'Korea, South', 'Laos', 'Moldova', 'North Macedonia', 'Russia',
       'Syria', 'Tanzania', 'US', 'United Kingdom', 'Venezuela',
       'Vietnam'], dtype=object)